In [ ]:
# Hack to import from a parent directory
import sys
path = '..'
if path not in sys.path:
    sys.path.append(path)

In [1]:
import stan
import arviz

#PyData imports
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
#Data to be passed to Stan.

temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.
prior_scale_factor = 0.25
obs_error_scale = 0.1

#Parameter prior means
u_Q_ref_mean = 0.25
Q_mean = 0.001
a_MSA_mean = 0.5
K_DE_mean = 1000
K_UE_mean = 0.1
V_DE_ref_mean = 0.11
V_UE_ref_mean = 0.0044
Ea_V_DE_mean = 40
Ea_V_UE_mean = 30
r_M_mean = 0.0001667
r_E_mean = 0.0002
r_L_mean = 0.0004

